In [11]:
import zipfile, os
import pandas as pd
import numpy as np
from PIL import Image
from glob import glob
from datetime import datetime as dt

In [12]:
np.random.seed(1775)

In [13]:
pjoin = os.path.join
TRAIN_FOLDERS = glob('/data/rgbd_face_data/train/*')
TEST_FOLDERS = glob('/data/rgbd_face_data/test/*')

In [14]:
def make_img_pair(fpaths, identical=True):
    folder1 = folder2 = np.random.choice(fpaths)
    if not identical:
        while os.path.samefile(folder1, folder2):
            folder2 = np.random.choice(fpaths)
    rgb_path1 = rgb_path2 = np.random.choice(glob(pjoin(folder1, "*.bmp")))
    while os.path.samefile(rgb_path1, rgb_path2):
        rgb_path2 = np.random.choice(glob(pjoin(folder2, "*.bmp")))
    depth_path1 = rgb_path1[:-5] + "d.dat"
    depth_path2 = rgb_path2[:-5] + "d.dat"
    
    # process image 1
    rgb1 = Image.open(rgb_path1)
    rgb1.thumbnail((640,480))
    rgb1 = np.asarray(rgb1)[140:340, 220:420, :3]
    depth1 = pd.read_csv(depth_path1, sep='\t', header=None)
    depth1[(depth1 > 3000)|(depth1 == -1)] = 3000
    depth1 = depth1.values[140:340, 220:420]
    depth1 = (depth1 - np.mean(depth1))/np.max(depth1)
    rgbd1 = np.dstack((rgb1, depth1))
    
    # process image 2
    rgb2 = Image.open(rgb_path1)
    rgb2.thumbnail((640,480))
    rgb2 = np.asarray(rgb2)[140:340, 220:420, :3]
    depth2 = pd.read_csv(depth_path2, sep='\t', header=None)
    depth2[(depth2 > 3000)|(depth2 == -1)] = 3000
    depth2 = depth2.values[140:340, 220:420]
    depth2 = (depth2 - np.mean(depth2))/np.max(depth2)
    rgbd2 = np.dstack((rgb2, depth2))
    
    return np.array([rgbd1, rgbd2])

# Network
---

In [15]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout, Lambda, ELU, concatenate, GlobalAveragePooling2D, Input, BatchNormalization, SeparableConv2D, Subtract, concatenate
from keras.activations import relu, softmax
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.optimizers import Adam, RMSprop, SGD
from keras.regularizers import l2
from keras import backend as K
from keras.callbacks import TensorBoard
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [16]:
def euclidean_distance(inputs):
    assert len(inputs) == 2, 'Euclidean distance needs 2 inputs, {} given'.format(len(inputs))
    u, v = inputs
    return K.sqrt(K.sum((K.square(u - v)), axis=1, keepdims=True))
        

def contrastive_loss(y_true,y_pred):
    margin=1.
    return K.mean((1. - y_true) * K.square(y_pred) + y_true * K.square(K.maximum(margin - y_pred, 0.)))

def generator(batch_size, folders):
    while True:
        X = []
        y = []
        identical = True
        for _ in range(batch_size):
            X.append(make_img_pair(folders, identical=identical))
            y.append(np.array([0.]))
            identical = not identical
        X = np.asarray(X)
        y = np.asarray(y)
        yield [X[:,0],X[:,1]], y
        
def fire(x, squeeze=16, expand=64):
    x = Convolution2D(squeeze, (1,1), padding='valid')(x)
    x = Activation('relu')(x)
    
    left = Convolution2D(expand, (1,1), padding='valid')(x)
    left = Activation('relu')(left)
    
    right = Convolution2D(expand, (3,3), padding='same')(x)
    right = Activation('relu')(right)
    
    x = concatenate([left, right], axis=3)
    return x

In [17]:
img_input=Input(shape=(200,200,4))

x = Convolution2D(64, (5, 5), strides=(2, 2), padding='valid')(img_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

x = fire(x, squeeze=16, expand=16)

x = fire(x, squeeze=16, expand=16)

x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)


x = fire(x, squeeze=32, expand=32)

x = fire(x, squeeze=32, expand=32)

x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)


x = fire(x, squeeze=48, expand=48)

x = fire(x, squeeze=48, expand=48)

x = fire(x, squeeze=64, expand=64)

x = fire(x, squeeze=64, expand=64)

x = Dropout(0.2)(x)

x = Convolution2D(512, (1, 1), padding='same')(x)
out = Activation('relu')(x)


modelsqueeze = Model(img_input, out)

In [18]:
im_in = Input(shape=(200,200,4))

x1 = modelsqueeze(im_in)

x1 = Flatten()(x1)

x1 = Dense(512, activation="relu")(x1)
x1 = Dropout(0.2)(x1)
feat_x = Dense(128, activation="linear")(x1)
feat_x = Lambda(lambda  x: K.l2_normalize(x,axis=1))(feat_x)


model_top = Model(inputs = [im_in], outputs = feat_x)

model_top.summary()

im_in1 = Input(shape=(200,200,4))
im_in2 = Input(shape=(200,200,4))

feat_x1 = model_top(im_in1)
feat_x2 = model_top(im_in2)


lambda_merge = Lambda(euclidean_distance)([feat_x1, feat_x2])


model_final = Model(inputs = [im_in1, im_in2], outputs = lambda_merge)

model_final.summary()

adam = Adam(lr=0.001)

sgd = SGD(lr=0.001, momentum=0.9)

model_final.compile(optimizer=adam, loss=contrastive_loss)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 200, 200, 4)       0         
_________________________________________________________________
model_4 (Model)              (None, 11, 11, 512)       253952    
_________________________________________________________________
flatten_2 (Flatten)          (None, 61952)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               31719936  
_________________________________________________________________
dropout_4 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 128)               65664     
_________________________________________________________________
lambda_3 (Lambda)            (None, 128)               0         
Total para

In [ ]:
time = dt.strftime(dt.now(), format='%Y-%m-%d_%H:%M:%S')
tensorboard = TensorBoard(log_dir='/home/astewart/repos/faceid/logs/{}'.format(time))

outputs = model_final.fit_generator(generator(16, TRAIN_FOLDERS), 
                                    steps_per_epoch=30, epochs=100, verbose=1,
                                    validation_data=generator(4, TEST_FOLDERS), 
                                    validation_steps=20, callbacks=[tensorboard])
model_final.save('/home/astewart/repos/faceid/models/{}.hdf5'.format(time))

Epoch 1/100
30/30 [==============================] - 101s 3s/step - loss: 0.0646 - val_loss: 1.1127e-11
Epoch 2/100
30/30 [==============================] - 94s 3s/step - loss: 0.0145 - val_loss: 5.0250e-12
Epoch 3/100
30/30 [==============================] - 93s 3s/step - loss: 0.0091 - val_loss: 4.1515e-12
Epoch 4/100
30/30 [==============================] - 94s 3s/step - loss: 0.0062 - val_loss: 2.9868e-12
Epoch 5/100
30/30 [==============================] - 94s 3s/step - loss: 0.0035 - val_loss: 2.5099e-12
Epoch 6/100
30/30 [==============================] - 94s 3s/step - loss: 5.3506e-04 - val_loss: 1.2423e-14
Epoch 7/100
30/30 [==============================] - 95s 3s/step - loss: 1.2833e-05 - val_loss: 2.3769e-14
Epoch 8/100
30/30 [==============================] - 94s 3s/step - loss: 9.0104e-07 - val_loss: 2.2703e-14
Epoch 9/100
30/30 [==============================] - 94s 3s/step - loss: 1.3464e-07 - val_loss: 8.9505e-15
Epoch 10/100
30/30 [==============================] - 94